In [ ]:
p_storage_dir = "../../storage"

In [19]:
import pandas as pd
import re
from pathlib import Path

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

# Cursos

In [20]:
cursos = pd.read_csv(DA_STORAGE_DIR / "transformed/cursos.csv", delimiter=";")

cursos.head()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
0,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,967,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3683,3186.0,120.0,160.0,108.0,3574.0,6
1,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Meio Ambi...,968,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3684,2970.0,120.0,160.0,54.0,3304.0,6
2,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Vigilânci...,908,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3592,3240.0,120.0,160.0,54.0,3574.0,6
3,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Enfermage...,909,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3593,2547.0,120.0,0.0,0.0,2667.0,8
4,Câmpus Águas Lindas,Licenciatura em Ciências Biológicas,1174,Licenciatura,Presencial,Noturno,Anual,30,Superior,4316,2960.0,200.0,0.0,108.0,3268.0,8


# Substituição do nome do câmpus pela sigla

Para garantir que o curso seja ligado corretamente com o câmpus em que é/foi ofertado, o nome do câmpus foi substituído pela sigla.

In [21]:
for campus in cursos["campus"].unique():
    print(f"• {campus}")

• Câmpus Águas Lindas
• Câmpus Anápolis
• Câmpus Aparecida de Goiânia
• Câmpus Cidade de Goiás
• Câmpus Formosa
• Câmpus Goiânia
• Câmpus Goiânia Oeste
• Câmpus Inhumas
• Câmpus Itumbiara
• Câmpus Jataí
• Câmpus Luziânia
• Câmpus Senador Canedo
• Câmpus Uruaçu
• Câmpus Valparaíso
• Câmpus Virtual de Ead


In [22]:
campus_replacement_map = {
    "Câmpus Águas Lindas": "LIN",
    "Câmpus Anápolis": "ANA",
    "Câmpus Aparecida de Goiânia": "APA",
    "Câmpus Cidade de Goiás": "GOI",
    "Câmpus Formosa": "FOR",
    "Câmpus Goiânia": "GYN",
    "Câmpus Goiânia Oeste": "OES",
    "Câmpus Inhumas": "INH",
    "Câmpus Itumbiara": "ITU",
    "Câmpus Jataí": "JAT",
    "Câmpus Luziânia": "LUZ",
    "Câmpus Senador Canedo": "SEN",
    "Câmpus Uruaçu": "URU",
    "Câmpus Valparaíso": "VAL"
}

In [23]:
cursos["campus"].replace(campus_replacement_map, inplace=True)

In [24]:
for campus in cursos["campus"].unique():
    print(f"• {campus}")

• LIN
• ANA
• APA
• GOI
• FOR
• GYN
• OES
• INH
• ITU
• JAT
• LUZ
• SEN
• URU
• VAL
• Câmpus Virtual de Ead


# Exclusão dos cursos do câmpus EAD

Atualmente existem 5 cursos que estão com o campo câmpus listado como "Câmpus Virtual de Ead" mas em seu nome estão vinculados a câmpus presenciais e o seu formato consta como "Presencial". 

Temporariamente eles serão excluídos do dataset resultante do pré-processamento até ser tomada uma decisão em relação ao formato de inclusão desses cursos, atividade que será realizada junto ao orientador.

In [25]:
cursos.drop(cursos[cursos["campus"] == "Câmpus Virtual de Ead"].index, inplace=True)

cursos.tail()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
252,VAL,Especialização em Ensino na Educação Básica,1427,Especialização,Presencial,Vespertino,Semestral,30,Pós-Graduação Lato Sensu,5316,420.0,0.0,0.0,0.0,420.0,1
253,VAL,Técnico Integrado ao Ensino Médio em Automação...,965,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3681,3240.0,120.0,200.0,54.0,3614.0,6
254,VAL,Técnico Integrado ao Ensino Médio em Mecânica,966,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3682,3240.0,120.0,200.0,0.0,3560.0,6
255,VAL,Técnico Integrado ao Ensino Médio em Eletrotéc...,932,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3646,2160.0,240.0,200.0,0.0,2600.0,8
256,VAL,Licenciatura em Matemática,1014,Licenciatura,Presencial,Noturno,Anual,30,Superior,4422,2970.0,200.0,0.0,54.0,3224.0,8


Existem alguns cursos com o formato listado como "À Distância", uma solução para os cursos com o câmpus "Câmpus Virtual de Ead" seria pegar o campo câmpus deles e atualizar com a sigla do câmpus que é mencionado em seu nome e trocar o seu campo formato para "À distância".

In [26]:
cursos[cursos["formato"] == "À Distância"].loc[:, ["campus", "nome", "codigo", "formato"]]

,campus,nome,codigo,formato
18,APA,Especialização em Docência na Educação Profiss...,1078,À Distância
37,FOR,Especialização em Docência na Educação Profiss...,1080,À Distância
38,FOR,Especialização em Docência para a Educação Pro...,1231,À Distância
39,FOR,Especialização em Docência para a Educação Pro...,1470,À Distância
167,OES,Especialização em Alfabetização e Letramento n...,1533,À Distância
168,OES,Especialização em Docência na Educação Profiss...,1081,À Distância
169,OES,Especialização em Docência para a Educação Pro...,1230,À Distância
170,OES,Especialização em Docência para a Educação Pro...,1468,À Distância
176,OES,Licenciatura em Formação Pedagógica para Gradu...,1093,À Distância
182,INH,Especialização em Docência na Educação Profiss...,1083,À Distância


# Resumo dos Cursos

Hipotéticamente esse dataset contém inúmeros dados duplicados, pois em tese todos os cursos tem os mesmos dados, o que os diferencia é o câmpus em que ele é ofertado. 

Caso essa hipótese seja comprovada é possível reduzir o dataset para apenas alguns cursos e criar outro dataset relacionando os cursos com os câmpus em que são ofertados.

In [27]:
cursos[cursos["nome"] == "Técnico Integrado ao Ensino Médio em Edificações"].iloc[:, 2:]

,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
9,342,Integrado,Presencial,Integral,Anual,30,Ensino Médio,2988,3186.0,120.0,200.0,108.0,3614.0,6
21,512,Integrado,Presencial,Integral,Anual,30,Ensino Médio,2582,3186.0,120.0,200.0,108.0,3614.0,6
32,533,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3925,3186.0,120.0,200.0,54.0,3560.0,6
116,91,Integrado,Presencial,Matutino,Anual,30,Ensino Médio,2940,3267.0,120.0,400.0,0.0,3787.0,8
207,234,Integrado,Presencial,Integral,Anual,30,Ensino Médio,2563,3294.0,120.0,200.0,108.0,3722.0,6
223,663,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3008,3294.0,120.0,200.0,108.0,3722.0,6
245,99,Integrado,Presencial,Integral,Anual,30,Ensino Médio,2532,3294.0,120.0,200.0,108.0,3722.0,6


Pelo que pôde ser observado, um mesmo curso tem diferentes propriedades, logo minha hipótese foi invalidada.

In [28]:
is_cursos_unique = cursos["codigo"].nunique() == cursos["codigo"].count()

if is_cursos_unique:
    print("Todos os cursos tem um código único, que pode ser usado como uma chave para identificá-los.")
else:
    print("Os cursos não tem um código único, logo não podem ser identificados unicamente.")

Todos os cursos tem um código único, que pode ser usado como uma chave para identificá-los.


# Exportação dos dados pré-processados

In [29]:
df_final = cursos

In [30]:
df_final.head()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
0,LIN,Técnico Integrado ao Ensino Médio em Análises ...,967,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3683,3186.0,120.0,160.0,108.0,3574.0,6
1,LIN,Técnico Integrado ao Ensino Médio em Meio Ambi...,968,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3684,2970.0,120.0,160.0,54.0,3304.0,6
2,LIN,Técnico Integrado ao Ensino Médio em Vigilânci...,908,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3592,3240.0,120.0,160.0,54.0,3574.0,6
3,LIN,Técnico Integrado ao Ensino Médio em Enfermage...,909,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3593,2547.0,120.0,0.0,0.0,2667.0,8
4,LIN,Licenciatura em Ciências Biológicas,1174,Licenciatura,Presencial,Noturno,Anual,30,Superior,4316,2960.0,200.0,0.0,108.0,3268.0,8


In [31]:
export_path = Path(DA_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    df_final.to_csv(export_path / "cursos.csv", sep=";", index=False)
except Exception as e:
    print(str(e))

In [37]:
cursos["nome"].unique()

array(['Técnico Integrado ao Ensino Médio em Análises Clínicas',
       'Técnico Integrado ao Ensino Médio em Meio Ambiente',
       'Técnico Integrado ao Ensino Médio em Vigilância em Saúde',
       'Técnico Integrado ao Ensino Médio em Enfermagem na Modalidade de Educação de Jovens e Adultos',
       'Licenciatura em Ciências Biológicas',
       'Bacharelado em Ciência da Computação',
       'Bacharelado em Engenharia Civil da Mobilidade',
       'Especialização em Inteligência Artificial Aplicada',
       'Técnico Integrado ao Ensino Médio em Comércio Exterior',
       'Técnico Integrado ao Ensino Médio em Edificações',
       'Técnico Integrado ao Ensino Médio em Química',
       'Técnico Integrado ao Ensino Médio em Secretaria Escolar na Modalidade de Educação de Jovens e Adultos',
       'Técnico Integrado ao Ensino Médio em Transporte de Cargas na Modalidade de Educação de Jovens e Adultos',
       'Licenciatura em Ciências Sociais', 'Licenciatura em Química',
       'Mestrado P